# Our modeling approach

- We fully characterize the existence of a 1-round protocol by a Mixed Integer Linear Program.


In [7]:
from sage.all import *
import itertools
import random
from sage.sat.solvers.satsolver import SAT
from sage.sat.solvers.cryptominisat import CryptoMiniSat
from sage.misc.temporary_file import atomic_write
import copy
import time

In [8]:
solver = SAT(solver="LP")
solver.add_clause((-1,2))
solver.add_clause((1,3))
solution = solver()
print ' solution =',solution
lst = ['play', 'ground']
print 'ground' in lst

m = matrix(QQ, [[1,2],[4,2],[11,3]])
print 'm = ',m
x = []
for v in m:
    x.append(v)
    
x = matrix(QQ, x)
print 'x = ',x
 
pr = Permutations(20).random_element()
print pr, pr[4], pr[7]


v = vector(QQ, [2,3,4])
print 'len = ',len(v)

min(5, 44)

l = [1,2,3,4,5]
print l [2:4]
(5 == 5) + 0


 solution = [None, False, False, True]
True
m =  [ 1  2]
[ 4  2]
[11  3]
x =  [ 1  2]
[ 4  2]
[11  3]
[14, 5, 11, 2, 19, 20, 4, 12, 7, 13, 17, 18, 16, 6, 1, 9, 3, 10, 15, 8] 19 12
len =  3
[3, 4]


1

In [9]:
# TODO: check correctness etc
def get_at(tup, a, l):
    """Given 2*l elements flattened into a tuple, pick 1 out of each pair according to array of bits."""
    assert len(tup) == 2*l
    assert len(a) == l
    for bit in a:
        assert bit in (0, 1)

    # tup structure is: $(t_{0,0}, t_{0,1}, t_{1,0}, t_{1,1}, t_{2,0}, t_{2,1})$
    return tuple(tup[2*i + a[i]] for i in range(l))

assert get_at((3, 13, 85, 95), (0, 1), 2) == (3, 95)

In [34]:
 
def tuples_fixed_proj(l, proj_ind, proj_val):
    tuples = list()
    x = [0 for i in range(2*l)]
    for i in range(l):
        x[i*2 + proj_ind[i]] = proj_val[i]
    for g in itertools.product(range(2), repeat = l):
        for i in range(l):
            x[i*2 + 1 - proj_ind[i]] = g[i]
        tuples.append(tuple(x[:]))
    return tuples     

                            
def get_rand_tup(len):
    return tuple([sage.misc.prandom.choice(range(2)) for i in range(len)])
                

# i = -1 , search for an index
# i = -2 : auxiliary for relaxed mode
# [1,0,0]
# [0,1,0]
# [0,0,1]
# [0,0,0]

def safe_insert(val_l, ind_l, (i,ind), v):
  #  print 'safe-inserting ',val_l, ind_l, (i,ind), v
    to_search = []
    if i >= 0:
        to_search = [i]
    elif i == -1:
        to_search = [j for j in range(3)]
    elif i == -2:
        val_l[ind_l.index(ind)] = v
        return
    
    for j in to_search:
        if (j,ind) in ind_l:
            val_l[ind_l.index((j,ind))] = v
      #      print 'inserted !',(i,ind)


def find_set_client_vars(l, p):
    
    l_list = [[],[],[]]
    for y in range(3):
        for a in itertools.product(range(2), repeat = l):
            for g in itertools.product(range(2), repeat = l):
                for z in range(2):
                    l_list[y].append((y,a,g,z))
                    
  #  print 'initial = '
  #  for y in range(3):
  #      print 'list[',y,']=', l_list[y]

    for (x,t) in p:            
        for y in range(3):
            # h == y
            for (h,a,g,b) in l_list[y]:
                if ((b == 0) and (y == x)) or ((b == 1) and (y != x)) and (get_at(t,a,l) == g):
                    l_list[y].remove((h,a,g,b)) 
    
    
    print 'remaining vars: '
    for y in range(3):
        print 'list[',y,']='
        for j in l_list[y]:
            print j
    
    
def find_set_server_vars(l, p0):
    try:
  #      print 'How much is server variable space limited by this solution & correctness?'
  #      print '|List1| = ',len(p0)
  #      print 'List=',p0
        p1 = []
        for i in range(3):                              
            for t in itertools.product(range(2), repeat = 2*l):
                p1.append((i,t))

        for (y,(a,g,b)) in p0:
            for x in range(3):
                if ((b == 0) and (y == x)) or ((b == 1) and (y != x)):
                    all = tuples_fixed_proj(l, a, g)
                    for t in all:
                        try:
                #            print 'removing ',(x, t)
                #            print 'Due to ',(y, (a,g,b))
                            p1.remove((x, t))
                        except Exception as e:
                            pass

        print 'number of remaining server values =',len(p1)
        if (len(p1) > 0):
            print 'server variables are ',p1
    except: 'Uncaught exception!'    


def permute_rows(m, b):
    
    nrows = m.nrows()
    perm = Permutations(nrows).random_element()
    pm = [row for row in m]
    pb = [v for v in b]
    
    new_m = matrix(QQ, [pm[perm[i] - 1] for i in range(nrows)])
    
    new_b = vector(QQ, [pb[perm[i] - 1] for i in range(nrows)])
    
    return (new_m, new_b)
    
    
    
def get_full_rank_sub(m, b):
   
    nrows = m.nrows()
    new_mat = []
    new_b = []
    
 #   print 'into get_full_rank_sub',m.nrows(), m.ncols()
    
    for (i,row) in enumerate(m):
        so_far = (matrix(QQ, new_mat)).transpose()
        try:
            so_far\row
        except Exception as e:
#            print 'at full rank sub Exception = ',e,i
            new_mat.append(row)
            new_b.append(b[i])
        
    new_mat = matrix(QQ, new_mat)
    new_b = vector(new_b)
    
    return (new_mat, new_b)


def print_matrix(m):

    if m.ncols < 37:
        print m
        return
    
    per_row = 30
    for row in m:
        n = ceil(QQ(m.nrows())/per_row)
        for i in range(n):
            ran = min(per_row, m.nrows() - i * per_row)
            print [row[i * per_row + j] for j in range(ran)]
        print '\n'    
          

def non_zeros(m):            
    return sum(sum(1 for j in row if abs(j) != 0) for row in m)        
        
def find_small_serv(l, out_of = 3):
    
    b = []
    constraints = []
    t_set = []
    t_part = [[],[],[]]
    
    for t in itertools.product(range(2), repeat = 2*l):
        i = sage.misc.prandom.choice(range(out_of))
        if (i < 3):
            t_set.append((i,t))
            t_part[i].append(t)
                
    for i in range(3):
        if len(t_part[i]) == 0:
            print 'failed attempt'
            return -1

    base_len = len(t_set)
    # add auxiliary variables
    
    for a in itertools.product(range(2), repeat = l):
        for y in range(3):
            for z in range(2):
                for g in itertools.product(range(2), repeat = l):
                    t_set.append((a,y,z,g))

    
    t_len = len(t_set)
    
#    print 'length = ',(base_len, t_len)
#    for (i,v) in enumerate(t_set):
#        print v,
#        if (i % 5 == 0):
#            print '\n'
    
    
    # add distribution constraints
    for x in range(3):
        cur_row = [0 for i in range(t_len)]
        for (i,t) in t_set[0:base_len]:
            if (x == i):
                cur_row[t_set.index((i,t))] = 1
        constraints.append(cur_row)
        b.append(1)
    
    
    for a in itertools.product(range(2), repeat = l): 
        for y in range(3):
            cur_row = [0 for i in range(t_len)]
            for g in itertools.product(range(2), repeat = l):
                cur_row[t_set.index((a,y,1,g))] = 1
                cur_row[t_set.index((a,y,0,g))] = -1
            constraints.append(cur_row)    
            b.append(0)

    
    for a in itertools.product(range(2), repeat = l):
        cur_row = [0 for i in range(t_len)]    
        for y in range(3):
            for g in itertools.product(range(2), repeat = l):
                cur_row[t_set.index((a,y,0,g))] = 1
            constraints.append(cur_row)    
            b.append(1)    
    
    
    # add security against malicious clients
    for a in itertools.product(range(2), repeat = l):
        for x in range(3):
            for g in itertools.product(range(2), repeat = l):
                ts = tuples_fixed_proj(l, a, g)
                cur_row = [0 for i in range(t_len)]
                for t in ts:
                    if t in t_part[x]:
                        cur_row[t_set.index((x, t))] = 1
                for y in range(3):
                    z = 0 + (x == y)
                    cur_row[t_set.index((a,y,z,g))] = -1 
                constraints.append(cur_row)
                b.append(0)
    
    
    m_constr = matrix(QQ,constraints)
    right_side = vector(QQ,b)
    
    print 'basic facts about the distribution: ', 'rank\ncols = ',m_constr.rank() ,m_constr.ncols()
    
    solver = MixedIntegerLinearProgram(maximization=True, solver='ppl')
    prot = solver.new_variable(nonnegative=True)
    aux = solver.new_variable(nonnegative=True)
        
    for (i,cur_row) in enumerate(constraints):
        solver.add_constraint(sum(cur_row[j]*prot[j] for j in range(t_len)) == b[i])
        
    try:    
        solver.solve()  
   #     print 'solution = '
   #     print 'main variables = ' 
        main_vars = [solver.get_values(prot[j]) for j in range(base_len)]
        
        
   #     for (i,j) in enumerate(main_vars):
   #         if j != 0:
   #             print (t_set[i],j)
        
        new_list = []
        for (i,j) in enumerate(main_vars):
            if j != 0:
                new_list.append(t_set[i])
         
        print '|new_list|=',len(new_list)
        
        find_set_client_vars(l, new_list)   
  #      print 'aux variables = '
  #      print [solver.get_values(prot[j]) for j in range(base_len,t_len)]
        
        
    except Exception as e:
        print 'Exception = ',e 
    
    
# Only captures client constraints + requirement that each b is assigned to a single y.   
# Attempts to model the ikops case. For each a - the set of `surviving' (g,z)'s is one of some 3 options.
# This is due to the symmetry of IKOPS.

def test_linear_solutions(l, up_to = 3, out_of = 3, verbose = 0):
    
        b = []
        constraints = []
        
        if verbose:
            print 'generating constraints ...'
        
        g_distr = [[],[],[]]
        
        for i in range(3):
            for g in itertools.product(range(2), repeat = l):
                j = sage.misc.prandom.choice(range(out_of))
                # the zeros exception is not necessary
                if (j < up_to) or (g == zeros):
                    g_distr[i].append((g,0))
                    g_distr[i].append((g,1))
                else:
                    # approximate that 2/3 are 0's, and 1/3 are 1's. Could change as well 
                    h = sage.misc.prandom.choice(range(3))
                    z = 0
                    if h == 2:
                        z = 1
                    g_distr[i].append((g,z))
                
        
        a_set = []
        for a in itertools.product(range(2), repeat = l):
            i = sage.misc.prandom.choice(range(3))
            for (g,z) in g_distr[i]:
                a_set.append((i,(a,g,z)))
        
        
        if verbose:
            for i in range(3):
                print 'g_distr[',i,']=', g_distr[i]
  

        a_set_len = len(a_set) 
        if verbose:
            print '|List0|=', a_set_len
        
        # well-formed distribution
            
        for i in range(3):
            cur_row = [0 for j in range(a_set_len)]
            for (j, (a, g, z)) in a_set:
                if (j == i) and (g == zeros):
                    cur_row[a_set.index((j, (a, g, z)))] = 1   
                
            constraints.append(cur_row)
            b.append(1) 
            
        solver = MixedIntegerLinearProgram(maximization=True, solver='ppl')
        prot = solver.new_variable(nonnegative=True)

        # require equiality of distributions
        for i in range(3):
            for a in itertools.product(range(2), repeat = l):
                for g in itertools.product(range(2), repeat = l): 
                    if g != zeros:
                        cur_row = [0 for j in range(a_set_len)]
                        # We are guaranteed that a is not completely out of the 
                        # set. If it was, eccidentally, this will make the set unsolvable
                        safe_insert(cur_row, a_set, (-1, (a, zeros, 0)), 1)
                        safe_insert(cur_row, a_set, (-1, (a, zeros, 1)), 1)
                        safe_insert(cur_row, a_set, (-1, (a, g, 0)), -1)
                        safe_insert(cur_row, a_set, (-1, (a, g, 1)), -1)
                        b.append(0)
                        
                        constraints.append(cur_row)

        
        # require security against a malicious server
        
        extra_constraints = []
        extra_b = []
        
        for t in itertools.product(range(2), repeat = 2*l):
            cur_row = [0 for i in range(a_set_len)]
            for a in itertools.product(range(2), repeat = l):
                safe_insert(cur_row, a_set, (-1, (a, get_at(t,a,l), 1)), 1)
            constraints.append(cur_row)
            b.append(1)
                
        
        m_constr = matrix(QQ,constraints)
        right_side = vector(QQ,b)
        
     #   print 'constraints = '
     #   print m_constr
     #   print 'right side = ', right_side
        
        
        solver = MixedIntegerLinearProgram(maximization=True, solver='ppl')
        prot = solver.new_variable(nonnegative=True)
        
        
        t_list = [a for a in itertools.product(range(2), repeat = 2*l)]
        
        if verbose:
            print 'Created linear system of rank',m_constr.rank(),'/',m_constr.nrows(), m_constr.ncols(),' vs ',a_set_len
        #    print m_constr
        try:
            v = m_constr\right_side
#            ker = m_constr.right_kernel()        
            print 'solution for linear system = ',v

            print 'len(solution) = ',len(v)
            nz = sum(1 for x in v if abs(x) > 0)
            
            print 'non-zeros ', nz
            
            if verbose:
                print '==================================================V'
                print 'Now looking for a well-formed solution, not just any'
            
            for (i,cur_row) in enumerate(constraints):
                solver.add_constraint(sum(cur_row[j]*prot[j] for j in range(a_set_len)) == b[i])
             
            
            solver.solve()  
            
            print 'Prot variables = '
            prot_l = [(a_set[i],solver.get_values(prot[i])) for i in range(a_set_len) if solver.get_values(prot[i]) > 0]
            for (i,x) in enumerate(prot_l):
                print x,
                if (i%3 == 0):
                    print '\n'
            nz = sum([1 for i in range(a_set_len) if solver.get_values(prot[i]) > 0])
            
            print 'overall ',nz,a_set_len
             
            find_set_server_vars([x for (x,y) in prot_l if y > 0],l)
            
            print '==================================================V'
        except Exception as e:
            print 'Exception = ',e
            if verbose:
                print '==================================================X'
            return -1


# global variables

my_eps = QQ(1/10000)

In [35]:
# main     
import time 


def main(dim, nsteps, up_to, lim = 3, relaxed  = 1):

    report_granularity = 1
    
    for step in range(nsteps):
        if (step % report_granularity) == 0:
            print 'STEP # ',step
            t0 = time.time()

        try:
   #         find_small_serv(l, lim)
            test_linear_solutions(l, up_to, lim, 1)

        finally:
            if (step % report_granularity) == 0:
                print 'time =', time.time() - t0,'\n'



In [6]:
for l in range(4,5):
    for lim in range(16,17):
        print 'l,lim = ',l,lim
        print '!!!!!!!!!!!!!!!!!!!!!!!!!!'
        zeros = tuple((0 for i in range(l)))
        main (l, 1, lim - 2, lim)

l,lim =  4 16
!!!!!!!!!!!!!!!!!!!!!!!!!!
STEP #  0
generating constraints ...
g_distr[ 0 ]= [((0, 0, 0, 0), 0), ((0, 0, 0, 0), 1), ((0, 0, 0, 1), 0), ((0, 0, 0, 1), 1), ((0, 0, 1, 0), 0), ((0, 0, 1, 0), 1), ((0, 0, 1, 1), 0), ((0, 0, 1, 1), 1), ((0, 1, 0, 0), 0), ((0, 1, 0, 0), 1), ((0, 1, 0, 1), 0), ((0, 1, 1, 0), 0), ((0, 1, 1, 0), 1), ((0, 1, 1, 1), 0), ((0, 1, 1, 1), 1), ((1, 0, 0, 0), 0), ((1, 0, 0, 0), 1), ((1, 0, 0, 1), 0), ((1, 0, 0, 1), 1), ((1, 0, 1, 0), 0), ((1, 0, 1, 0), 1), ((1, 0, 1, 1), 0), ((1, 0, 1, 1), 1), ((1, 1, 0, 0), 0), ((1, 1, 0, 0), 1), ((1, 1, 0, 1), 0), ((1, 1, 0, 1), 1), ((1, 1, 1, 0), 0), ((1, 1, 1, 1), 0), ((1, 1, 1, 1), 1)]
g_distr[ 1 ]= [((0, 0, 0, 0), 0), ((0, 0, 0, 0), 1), ((0, 0, 0, 1), 0), ((0, 0, 1, 0), 0), ((0, 0, 1, 0), 1), ((0, 0, 1, 1), 0), ((0, 0, 1, 1), 1), ((0, 1, 0, 0), 0), ((0, 1, 0, 0), 1), ((0, 1, 0, 1), 0), ((0, 1, 0, 1), 1), ((0, 1, 1, 0), 0), ((0, 1, 1, 0), 1), ((0, 1, 1, 1), 0), ((0, 1, 1, 1), 1), ((1, 0, 0, 0), 0), ((1, 0, 0, 0), 1),

In [9]:
for l in range(4,5):
    for lim in range(16,17):
        print 'l,lim = ',l,lim
        print '!!!!!!!!!!!!!!!!!!!!!!!!!!'
        zeros = tuple((0 for i in range(l)))
        main (l, 1, lim - 3, lim)

l,lim =  4 16
!!!!!!!!!!!!!!!!!!!!!!!!!!
STEP #  0
generating constraints ...
g_distr[ 0 ]= [((0, 0, 0, 0), 0), ((0, 0, 0, 0), 1), ((0, 0, 0, 1), 1), ((0, 0, 1, 0), 1), ((0, 0, 1, 1), 0), ((0, 1, 0, 0), 0), ((0, 1, 0, 0), 1), ((0, 1, 0, 1), 0), ((0, 1, 1, 0), 0), ((0, 1, 1, 0), 1), ((0, 1, 1, 1), 0), ((0, 1, 1, 1), 1), ((1, 0, 0, 0), 0), ((1, 0, 0, 0), 1), ((1, 0, 0, 1), 0), ((1, 0, 1, 0), 0), ((1, 0, 1, 0), 1), ((1, 0, 1, 1), 0), ((1, 0, 1, 1), 1), ((1, 1, 0, 0), 0), ((1, 1, 0, 0), 1), ((1, 1, 0, 1), 0), ((1, 1, 0, 1), 1), ((1, 1, 1, 0), 0), ((1, 1, 1, 1), 0), ((1, 1, 1, 1), 1)]
g_distr[ 1 ]= [((0, 0, 0, 0), 0), ((0, 0, 0, 0), 1), ((0, 0, 0, 1), 0), ((0, 0, 0, 1), 1), ((0, 0, 1, 0), 0), ((0, 0, 1, 0), 1), ((0, 0, 1, 1), 0), ((0, 0, 1, 1), 1), ((0, 1, 0, 0), 0), ((0, 1, 0, 1), 0), ((0, 1, 0, 1), 1), ((0, 1, 1, 0), 0), ((0, 1, 1, 0), 1), ((0, 1, 1, 1), 0), ((0, 1, 1, 1), 1), ((1, 0, 0, 0), 0), ((1, 0, 0, 0), 1), ((1, 0, 0, 1), 0), ((1, 0, 0, 1), 1), ((1, 0, 1, 0), 0), ((1, 0, 1, 0), 1),

In [36]:
for l in range(4,5):
    for lim in range(3,4):
        print 'l,lim = ',l,pow(2,l) - lim,'/',pow(2,l)
        print '!!!!!!!!!!!!!!!!!!!!!!!!!!'
        zeros = tuple((0 for i in range(l)))
        main (l, 1, pow(2,l) - lim, pow(2,l))

l,lim =  4 13 / 16
!!!!!!!!!!!!!!!!!!!!!!!!!!
STEP #  0
generating constraints ...
g_distr[ 0 ]= [((0, 0, 0, 0), 0), ((0, 0, 0, 0), 1), ((0, 0, 0, 1), 0), ((0, 0, 0, 1), 1), ((0, 0, 1, 0), 0), ((0, 0, 1, 0), 1), ((0, 0, 1, 1), 0), ((0, 0, 1, 1), 1), ((0, 1, 0, 0), 0), ((0, 1, 0, 0), 1), ((0, 1, 0, 1), 0), ((0, 1, 1, 0), 0), ((0, 1, 1, 0), 1), ((0, 1, 1, 1), 0), ((0, 1, 1, 1), 1), ((1, 0, 0, 0), 0), ((1, 0, 0, 0), 1), ((1, 0, 0, 1), 1), ((1, 0, 1, 0), 0), ((1, 0, 1, 0), 1), ((1, 0, 1, 1), 0), ((1, 0, 1, 1), 1), ((1, 1, 0, 0), 0), ((1, 1, 0, 0), 1), ((1, 1, 0, 1), 1), ((1, 1, 1, 0), 0), ((1, 1, 1, 0), 1), ((1, 1, 1, 1), 0), ((1, 1, 1, 1), 1)]
g_distr[ 1 ]= [((0, 0, 0, 0), 0), ((0, 0, 0, 0), 1), ((0, 0, 0, 1), 0), ((0, 0, 0, 1), 1), ((0, 0, 1, 0), 0), ((0, 0, 1, 0), 1), ((0, 0, 1, 1), 0), ((0, 0, 1, 1), 1), ((0, 1, 0, 0), 0), ((0, 1, 0, 0), 1), ((0, 1, 0, 1), 0), ((0, 1, 0, 1), 1), ((0, 1, 1, 0), 0), ((0, 1, 1, 0), 1), ((0, 1, 1, 1), 0), ((0, 1, 1, 1), 1), ((1, 0, 0, 0), 0), ((1, 0, 0, 0)

In [14]:
# 440 - No solutions even to the regular system
# 485, 486, 481, 454 - solution with 48 0's

for l in range(4,5):
    for lim in range(3,4):
        print 'l,lim = ',l,pow(2,l) - lim,'/',pow(2,l)
        print '!!!!!!!!!!!!!!!!!!!!!!!!!!'
        zeros = tuple((0 for i in range(l)))
        main (l, 5, pow(2,l) - lim, pow(2,l))

l,lim =  4 13 / 16
!!!!!!!!!!!!!!!!!!!!!!!!!!
STEP #  0
generating constraints ...
g_distr[ 0 ]= [((0, 0, 0, 0), 0), ((0, 0, 0, 0), 1), ((0, 0, 0, 1), 0), ((0, 0, 0, 1), 1), ((0, 0, 1, 0), 0), ((0, 0, 1, 0), 1), ((0, 0, 1, 1), 0), ((0, 0, 1, 1), 1), ((0, 1, 0, 0), 0), ((0, 1, 0, 0), 1), ((0, 1, 0, 1), 0), ((0, 1, 0, 1), 1), ((0, 1, 1, 0), 0), ((0, 1, 1, 0), 1), ((0, 1, 1, 1), 0), ((0, 1, 1, 1), 1), ((1, 0, 0, 0), 0), ((1, 0, 0, 0), 1), ((1, 0, 0, 1), 0), ((1, 0, 0, 1), 1), ((1, 0, 1, 0), 0), ((1, 0, 1, 0), 1), ((1, 0, 1, 1), 0), ((1, 0, 1, 1), 1), ((1, 1, 0, 0), 0), ((1, 1, 0, 0), 1), ((1, 1, 0, 1), 0), ((1, 1, 0, 1), 1), ((1, 1, 1, 0), 0), ((1, 1, 1, 1), 0), ((1, 1, 1, 1), 1)]
g_distr[ 1 ]= [((0, 0, 0, 0), 0), ((0, 0, 0, 0), 1), ((0, 0, 0, 1), 0), ((0, 0, 0, 1), 1), ((0, 0, 1, 0), 0), ((0, 0, 1, 0), 1), ((0, 0, 1, 1), 0), ((0, 1, 0, 0), 0), ((0, 1, 0, 0), 1), ((0, 1, 0, 1), 0), ((0, 1, 0, 1), 1), ((0, 1, 1, 0), 0), ((0, 1, 1, 0), 1), ((0, 1, 1, 1), 0), ((0, 1, 1, 1), 1), ((1, 0, 0, 0)

==================================================V
Now looking for a well-formed solution, not just any
The optimal error sum =  0
Prot variables = 
((1, ((0, 0, 0, 1), (0, 0, 0, 0), 0)), 1) 

((1, ((0, 0, 0, 1), (0, 0, 0, 1), 0)), 1) ((1, ((0, 0, 0, 1), (0, 0, 1, 0), 1)), 1) ((1, ((0, 0, 0, 1), (0, 0, 1, 1), 0)), 1) 

((1, ((0, 0, 0, 1), (0, 1, 0, 0), 0)), 1) ((1, ((0, 0, 0, 1), (0, 1, 0, 1), 0)), 1) ((1, ((0, 0, 0, 1), (0, 1, 1, 0), 0)), 1) 

((1, ((0, 0, 0, 1), (0, 1, 1, 1), 0)), 1) ((1, ((0, 0, 0, 1), (1, 0, 0, 0), 0)), 1) ((1, ((0, 0, 0, 1), (1, 0, 0, 1), 0)), 1) 

((1, ((0, 0, 0, 1), (1, 0, 1, 0), 1)), 1) ((1, ((0, 0, 0, 1), (1, 0, 1, 1), 0)), 1) ((1, ((0, 0, 0, 1), (1, 1, 0, 0), 0)), 1) 

((1, ((0, 0, 0, 1), (1, 1, 0, 1), 0)), 1) ((1, ((0, 0, 0, 1), (1, 1, 1, 0), 0)), 1) ((1, ((0, 0, 0, 1), (1, 1, 1, 1), 0)), 1) 

((2, ((1, 0, 0, 0), (0, 0, 0, 0), 0)), 1) ((2, ((1, 0, 0, 0), (0, 0, 0, 1), 0)), 1) ((2, ((1, 0, 0, 0), (0, 0, 1, 0), 0)), 1) 

((2, ((1, 0, 0, 0), (0, 0, 1, 1), 0)),

Created linear system of rank 324 / 979 428  vs  428
solution for linear system =  (1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 2, -1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

KeyboardInterrupt: 

In [16]:
for l in range(3,4):
    for lim in range(1,2):
        print 'l,lim = ',l,pow(2,l) - lim,'/',pow(2,l)
        print '!!!!!!!!!!!!!!!!!!!!!!!!!!'
        zeros = tuple((0 for i in range(l)))
        main (l, 10, pow(2,l) - lim, pow(2,l))

l,lim =  3 7 / 8
!!!!!!!!!!!!!!!!!!!!!!!!!!
STEP #  0
generating constraints ...
g_distr[ 0 ]= [((0, 0, 0), 0), ((0, 0, 0), 1), ((0, 0, 1), 0), ((0, 0, 1), 1), ((0, 1, 0), 0), ((0, 1, 0), 1), ((0, 1, 1), 0), ((0, 1, 1), 1), ((1, 0, 0), 0), ((1, 0, 0), 1), ((1, 0, 1), 0), ((1, 0, 1), 1), ((1, 1, 0), 0), ((1, 1, 0), 1), ((1, 1, 1), 0), ((1, 1, 1), 1)]
g_distr[ 1 ]= [((0, 0, 0), 0), ((0, 0, 0), 1), ((0, 0, 1), 0), ((0, 0, 1), 1), ((0, 1, 0), 0), ((0, 1, 0), 1), ((0, 1, 1), 0), ((1, 0, 0), 0), ((1, 0, 0), 1), ((1, 0, 1), 0), ((1, 1, 0), 0), ((1, 1, 0), 1), ((1, 1, 1), 0), ((1, 1, 1), 1)]
g_distr[ 2 ]= [((0, 0, 0), 0), ((0, 0, 0), 1), ((0, 0, 1), 0), ((0, 0, 1), 1), ((0, 1, 0), 0), ((0, 1, 0), 1), ((0, 1, 1), 0), ((1, 0, 0), 0), ((1, 0, 0), 1), ((1, 0, 1), 0), ((1, 0, 1), 1), ((1, 1, 0), 0), ((1, 1, 0), 1), ((1, 1, 1), 0), ((1, 1, 1), 1)]
|List0|= 120
Created linear system of rank 86 / 235 120  vs  120
solution for linear system =  (0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0

The optimal error sum =  0
Prot variables = 
((1, ((0, 0, 0), (0, 0, 0), 1)), 1) 

((1, ((0, 0, 0), (0, 0, 1), 1)), 1) ((1, ((0, 0, 0), (0, 1, 0), 1)), 1) ((1, ((0, 0, 0), (0, 1, 1), 1)), 1) 

((1, ((0, 0, 0), (1, 0, 0), 0)), 1) ((1, ((0, 0, 0), (1, 0, 1), 0)), 1) ((1, ((0, 0, 0), (1, 1, 0), 0)), 1) 

((1, ((0, 0, 0), (1, 1, 1), 0)), 1) ((2, ((0, 0, 1), (0, 0, 0), 0)), 1) ((2, ((0, 0, 1), (0, 0, 1), 0)), 1) 

((2, ((0, 0, 1), (0, 1, 0), 0)), 1) ((2, ((0, 0, 1), (0, 1, 1), 0)), 1) ((2, ((0, 0, 1), (1, 0, 0), 0)), 1) 

((2, ((0, 0, 1), (1, 0, 1), 0)), 1) ((2, ((0, 0, 1), (1, 1, 0), 0)), 1) ((2, ((0, 0, 1), (1, 1, 1), 0)), 1) 

((0, ((0, 1, 0), (0, 0, 0), 0)), 1) ((0, ((0, 1, 0), (0, 0, 1), 0)), 1) ((0, ((0, 1, 0), (0, 1, 0), 0)), 1) 

((0, ((0, 1, 0), (0, 1, 1), 0)), 1) ((0, ((0, 1, 0), (1, 0, 0), 1)), 1) ((0, ((0, 1, 0), (1, 0, 1), 1)), 1) 

((0, ((0, 1, 0), (1, 1, 0), 1)), 1) ((0, ((0, 1, 0), (1, 1, 1), 1)), 1) overall  24 128
==================================================V
time = 

((0, ((0, 0, 0), (0, 0, 1), 1)), 1) ((0, ((0, 0, 0), (0, 1, 0), 0)), 1) ((0, ((0, 0, 0), (0, 1, 1), 1)), 1) 

((0, ((0, 0, 0), (1, 0, 0), 1)), 1) ((0, ((0, 0, 0), (1, 0, 1), 0)), 1) ((0, ((0, 0, 0), (1, 1, 0), 1)), 1) 

((0, ((0, 0, 0), (1, 1, 1), 0)), 1) ((1, ((0, 0, 1), (0, 0, 0), 0)), 1) ((1, ((0, 0, 1), (0, 0, 1), 0)), 1) 

((1, ((0, 0, 1), (0, 1, 0), 0)), 1) ((1, ((0, 0, 1), (0, 1, 1), 0)), 1) ((1, ((0, 0, 1), (1, 0, 0), 0)), 1) 

((1, ((0, 0, 1), (1, 0, 1), 0)), 1) ((1, ((0, 0, 1), (1, 1, 0), 0)), 1) ((1, ((0, 0, 1), (1, 1, 1), 0)), 1) 

((2, ((0, 1, 0), (0, 0, 0), 1)), 1) ((2, ((0, 1, 0), (0, 0, 1), 0)), 1) ((2, ((0, 1, 0), (0, 1, 0), 1)), 1) 

((2, ((0, 1, 0), (0, 1, 1), 0)), 1) ((2, ((0, 1, 0), (1, 0, 0), 0)), 1) ((2, ((0, 1, 0), (1, 0, 1), 1)), 1) 

((2, ((0, 1, 0), (1, 1, 0), 0)), 1) ((2, ((0, 1, 0), (1, 1, 1), 1)), 1) overall  24 122
==================================================V
time = 1.05962395668 

STEP #  9
generating constraints ...
g_distr[ 0 ]= [((0, 0, 0), 0),

In [31]:
for l in range(2,3):
    for lim in range(0,1):
        print 'l,lim = ',l,pow(2,l) - lim,'/',pow(2,l)
        print '!!!!!!!!!!!!!!!!!!!!!!!!!!'
        zeros = tuple((0 for i in range(l)))
        main (l, 10, pow(2,l) - lim, pow(2,l))

l,lim =  2 4 / 4
!!!!!!!!!!!!!!!!!!!!!!!!!!
STEP #  0
generating constraints ...
g_distr[ 0 ]= [((0, 0), 0), ((0, 0), 1), ((0, 1), 0), ((0, 1), 1), ((1, 0), 0), ((1, 0), 1), ((1, 1), 0), ((1, 1), 1)]
g_distr[ 1 ]= [((0, 0), 0), ((0, 0), 1), ((0, 1), 0), ((0, 1), 1), ((1, 0), 0), ((1, 0), 1), ((1, 1), 0), ((1, 1), 1)]
g_distr[ 2 ]= [((0, 0), 0), ((0, 0), 1), ((0, 1), 0), ((0, 1), 1), ((1, 0), 0), ((1, 0), 1), ((1, 1), 0), ((1, 1), 1)]
|List0|= 32
constraints = 
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1  0  0  0  0  0  0  1  1  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 1  1  0  0  0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 1  1 -1 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 1  1  0  0 -1 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 1  1  0  0  0  0 -1 -1  0  0  0  0  0  0  0

Created linear system of rank 23 / 55 32  vs  32
Exception =  matrix equation has no solutions
==================================================X
time = 0.0101478099823 

STEP #  7
generating constraints ...
g_distr[ 0 ]= [((0, 0), 0), ((0, 0), 1), ((0, 1), 0), ((0, 1), 1), ((1, 0), 0), ((1, 0), 1), ((1, 1), 0), ((1, 1), 1)]
g_distr[ 1 ]= [((0, 0), 0), ((0, 0), 1), ((0, 1), 0), ((0, 1), 1), ((1, 0), 0), ((1, 0), 1), ((1, 1), 0), ((1, 1), 1)]
g_distr[ 2 ]= [((0, 0), 0), ((0, 0), 1), ((0, 1), 0), ((0, 1), 1), ((1, 0), 0), ((1, 0), 1), ((1, 1), 0), ((1, 1), 1)]
|List0|= 32
constraints = 
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 1  1  0  0  0  0  0  0  1  1  0  0  0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1  0  0  0  0  0  0]
[ 1  1 -1 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 1  1  0  0 -1